#### Importar bibliotecas

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta, datetime
import time
import re

In [31]:
website = 'https://get.rnbastreams.com'
pagina = requests.get(website)
soup = BeautifulSoup(pagina.text,'html')

#### **Substituindo "22:00 EST" por x**

In [32]:
def remove_horarioEST(input_str):
    padrao = r'\s\d{1,2}:\d{2}\sEST\s'
    str = re.sub(padrao,'x', input_str)
    return str.strip()

In [34]:
# Antes
for jogos in soup.find_all("div", {"class":"competition"}):
  jogos = jogos.text.strip().replace("\n\n\n\n"," ").replace(" \n\n\n\n", " ").replace("\n\n\n", " ").replace("  In Progress  ", " x ").replace(" Live Streams","").replace("  Halftime  ", " x ")
  print(jogos)

Charlotte Hornets x Miami Heat
Detroit Pistons x Boston Celtics
Washington Wizards x Cleveland Cavaliers
Chicago Bulls x Oklahoma City Thunder
Memphis Grizzlies x Orlando Magic
Minnesota Timberwolves x Toronto Raptors
San Antonio Spurs x Houston Rockets
Phoenix Suns  22:00 EST  Dallas Mavericks
Los Angeles Lakers  22:30 EST  Sacramento Kings


In [35]:
# Depois
for jogos in soup.find_all("div", {"class":"competition"}):
  jogo_v2 = remove_horarioEST(jogos.text.strip().replace("\n\n\n\n"," ").replace(" \n\n\n\n", " ").replace("\n\n\n", " ").replace("  In Progress  ", " x ").replace(" Live Streams","").replace("  Halftime  ", " x "))
  print(jogo_v2)

Charlotte Hornets x Miami Heat
Detroit Pistons x Boston Celtics
Washington Wizards x Cleveland Cavaliers
Chicago Bulls x Oklahoma City Thunder
Memphis Grizzlies x Orlando Magic
Minnesota Timberwolves x Toronto Raptors
San Antonio Spurs x Houston Rockets
Phoenix Suns x Dallas Mavericks
Los Angeles Lakers x Sacramento Kings


In [38]:
jogos_live = []
# ------------------------- PÁGINA 01 -------------------------
for jogos in soup.find_all("div", {"class":"competition"}):
    jogo = remove_horarioEST(jogos.text.strip().replace("\n\n\n\n"," ").replace(" \n\n\n\n", " ").replace("\n\n\n", " ").replace("  In Progress  ", " x ").replace(" Live Streams","").replace("  Halftime  ", " x "))
    link_jogo = website + jogos.find("a")["href"] if jogos.find("a") else None
    horario = jogos.find("span", {"class": "competition-cell-status competition-cell-time"}).text.strip().replace(" EST","")
# ------------------------- PÁGINA 02 -------------------------
    pagina2 = requests.get(link_jogo)
    soup2 = BeautifulSoup(pagina2.text, 'html')
    resultDate = soup2.find(string=re.compile("EST")).text.split()
    dateGame = datetime.strptime(resultDate[0], '%Y-%m-%d').date()
    timeGame = datetime.strptime(resultDate[1], '%H:%M').time()
    jogos_live.append({'Jogos_Hoje': datetime.combine(dateGame,timeGame), 'jogo': jogo, 'link_jogo': link_jogo, 'Status': horario})

#### **Todos os jogos de hoje**

In [76]:
jogos_live

[{'Jogos_Hoje': datetime.datetime(2024, 10, 26, 19, 0),
  'jogo': 'Charlotte Hornets x Miami Heat',
  'link_jogo': 'https://get.rnbastreams.com/game/miami-heat-at-charlotte-hornets-6-live-stream ',
  'Status': 'In Progress'},
 {'Jogos_Hoje': datetime.datetime(2024, 10, 26, 19, 0),
  'jogo': 'Detroit Pistons x Boston Celtics',
  'link_jogo': 'https://get.rnbastreams.com/game/boston-celtics-at-detroit-pistons-4-live-stream ',
  'Status': 'In Progress'},
 {'Jogos_Hoje': datetime.datetime(2024, 10, 26, 19, 0),
  'jogo': 'Washington Wizards x Cleveland Cavaliers',
  'link_jogo': 'https://get.rnbastreams.com/game/cleveland-cavaliers-at-washington-wizards-4-live-stream ',
  'Status': 'In Progress'},
 {'Jogos_Hoje': datetime.datetime(2024, 10, 26, 20, 0),
  'jogo': 'Chicago Bulls x Oklahoma City Thunder',
  'link_jogo': 'https://get.rnbastreams.com/game/oklahoma-city-thunder-at-chicago-bulls-3-live-stream ',
  'Status': 'In Progress'},
 {'Jogos_Hoje': datetime.datetime(2024, 10, 26, 20, 0),
  

#### **O objetivo agora é saber quanto tempo falta para iniciar a partida**

In [77]:
data_agora = datetime.now().date()
hora_agora = datetime.now().time().replace(second=0, microsecond=0)
dataHoraAgora = datetime.combine(data_agora,hora_agora)

#### **Verficando o tempo restante para inicio da partida ou saber se o status está em "In Progress" ou "Halftime"**

In [78]:
while True:
    for jogoNBA in jogos_live:
        if jogoNBA['Jogos_Hoje'] > dataHoraAgora:
            inicioJogo = jogoNBA['Jogos_Hoje'] - datetime.combine(dataHoraAgora.date(), dataHoraAgora.time())
            print(jogoNBA['jogo'], '\n', jogoNBA['link_jogo'], '\n' f'Falta {inicioJogo} para inicio do jogo', '\n''---')
        elif jogoNBA['Status'] == 'In Progress':
            print(jogoNBA['jogo'], '\n', jogoNBA['link_jogo'], '\n', 'In Progress', '\n''---')
        elif jogoNBA['Status'] == 'Halftime':
            print(jogoNBA['jogo'], '\n', jogoNBA['link_jogo'], '\n', 'Halftime', '\n''---')
    break

Charlotte Hornets x Miami Heat 
 https://get.rnbastreams.com/game/miami-heat-at-charlotte-hornets-6-live-stream  
 In Progress 
---
Detroit Pistons x Boston Celtics 
 https://get.rnbastreams.com/game/boston-celtics-at-detroit-pistons-4-live-stream  
 In Progress 
---
Washington Wizards x Cleveland Cavaliers 
 https://get.rnbastreams.com/game/cleveland-cavaliers-at-washington-wizards-4-live-stream  
 In Progress 
---
Chicago Bulls x Oklahoma City Thunder 
 https://get.rnbastreams.com/game/oklahoma-city-thunder-at-chicago-bulls-3-live-stream  
 In Progress 
---
Memphis Grizzlies x Orlando Magic 
 https://get.rnbastreams.com/game/orlando-magic-at-memphis-grizzlies-4-live-stream  
 In Progress 
---
Minnesota Timberwolves x Toronto Raptors 
 https://get.rnbastreams.com/game/toronto-raptors-at-minnesota-timberwolves-3-live-stream  
 In Progress 
---
San Antonio Spurs x Houston Rockets 
 https://get.rnbastreams.com/game/houston-rockets-at-san-antonio-spurs-7-live-stream  
 In Progress 
---


#### **Mandando a notificação das partidas via Telegram**

In [ ]:
token =  "TOKEN"
chat_id = "CHAT_ID"

while True:
    for jogoNBA in jogos_live:
        if jogoNBA['Jogos_Hoje'] > dataHoraAgora:
            inicioJogo = jogoNBA['Jogos_Hoje'] - datetime.combine(dataHoraAgora.date(), dataHoraAgora.time())
            mensagem = f"{jogoNBA['jogo']}\n{jogoNBA['link_jogo']}\nFalta {inicioJogo} para início do jogo\n---"
        elif jogoNBA['Status'] == 'In Progress':
            mensagem = f"{jogoNBA['jogo']}\n{jogoNBA['link_jogo']}\nIn Progress\n---"
        elif jogoNBA['Status'] == 'Halftime':
            mensagem = f"{jogoNBA['jogo']}\n{jogoNBA['link_jogo']}\nHalftime\n---"

        # Envia a mensagem apenas se não estiver vazia
        if mensagem:
            url = f"https://api.telegram.org/bot{token}/sendMessage"
            params = {
                "chat_id": chat_id,
                "text": mensagem
            }
            response = requests.post(url, data=params)

            # Verificação do envio
            if response.ok:
                print("Mensagem enviada com sucesso!")
            else:
                print("Erro ao enviar a mensagem:", response.json())
    break